In [1]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 21.9 MB 138 kB/s
     |████████████████████████████████| 596 kB 3.1 MB/s 
     |████████████████████████████████| 102 kB 44.1 MB/s 
     |████████████████████████████████| 690 kB 35.2 MB/s 
     |████████████████████████████████| 4.7 MB 13.7 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 812 kB 22.5 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.2.0-py2.py3-none-any.whl size=261257 sha256=341c9d430386991e5f963fde2d86b2feab00efcc220cf4de8eb932241ab9238a
  Stored in directory: /tmp/pip-ephem-wheel-cache-7pn7dful/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=4f359304e428c5d5c3e6d93746024b7801835fcfe118230770b9db1cdfadef07
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940

In [2]:
# importing models
from pandas_profiling import ProfileReport
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import re

# Create a root variable to store the root url of maroof website
root = 'https://maroof.sa'
'''
each store in maroof has a page, this page url is a combination of the root
and the id number in maroof, for example: the url here https://maroof.sa/7855 
has two parts the first part is the root (https://maroof.sa) and the second 
part is the sore's id in maroof(7855). 
'''
# generate multiple store urls between the rage of 1000-1500 
stores_urls = []
for i in range(1000,1501): 
    stores_urls.append(root+"/"+str(i))

# create multiple empty containers to store the extracted data
store_names = []
busenss_typs = []
commercial_records = []
business_chanels = []
maroof_numbers = []
ratings = []
number_raters = []
about_stores = []
stores_websites = []
stores_accounts = []
maroof_numbers_websites = []
email_addresses = []
store_emails_ids = []

'''
The code below aims to extract data from HTML pages and then store every single
piece of information in one of the containers above to store all the data in 
one source (CSV file). 
'''
for url in stores_urls: 
  result = requests.get(url) # request html content for each store page in Maroof 
  soup_maroof = BeautifulSoup(result.content,'lxml')
  stores_website = soup_maroof.find('a',{'id': "websiteURLAnchor"})
  maroof_number = soup_maroof.find_all('span', {'class':'maarof-number'})
  accounts = soup_maroof.find_all('p',{'class' :"text-primary"})
  if maroof_number: 
      accounts.append(maroof_number[0])
  # Extracting all contact channels of each store 
  temp_account=[]
  if accounts:
    for account in accounts:
      clean = re.compile('<.*?>')
      account = re.sub(clean, '', str(account.contents[0]))
      temp_account.append(account)
      stores_accounts.append(set(temp_account))

  # Extracting websites 
  if stores_website: 
     clean = re.compile('<.*?>')
     stores_website = re.sub(clean, '', str(stores_website.contents[0]))
     stores_websites.append(stores_website)
     if maroof_number: 
       maroof_numbers_websites.append(maroof_number[0].text[-5:])
  # Extracting store names 
  text_primary = soup_maroof.find('h3',{'class': 'media-heading text-primary'})
  if text_primary:
    store_name = text_primary.contents[1].text
    store_names.append(str(store_name))
  # Extracting business types
  busenss_typ= soup_maroof.div.p
  if busenss_typ:
    busenss_typ = busenss_typ.contents[0]
    busenss_typs.append(str(busenss_typ))
  # Extracting السجل التجاري 
  divs = soup_maroof.find_all('div', class_='media-body media-body--width')
  for span in divs:
    commercial_record = span.find_all('span')[2].text
    if commercial_record == ' متجر إلكتروني  ':
      commercial_records.append(None)
    else:
     commercial_records.append(commercial_record)

    business_chanel = span.find_all('span')[-1].text
    business_chanels.append(business_chanel)
  # Extracting phone number 
  maroof_number = soup_maroof.find_all('span', {'class':'maarof-number'})
  if maroof_number: 
    maroof_numbers.append(maroof_number[0].text[-5:])
   
  # اسنخراج تقيم المتجر في موقع معروف 
  rating_num = soup_maroof.find_all('span', {'class':'rating-num'})
  if rating_num: 
    ratings.append(rating_num[1].text)
  # استخراج عدد المقيمين للمتجر في معروف 
  number_rater = soup_maroof.find('div', {'class':'h3 text-primary ma'})
  if number_rater: 
    number_raters.append(number_rater.text[:-9])
  # Brief info about what each store does/provides. 
  about_store = soup_maroof.find('div', {'class':'col-xs-12 withScroll'})
  if about_store: 
    clean = re.compile('<.*?\\>')
    about_store = re.sub(clean, '', str(about_store.contents[0]).replace("\r\n                ",""))
    about_stores.append(about_store)

# Extracting email address 
for account in stores_accounts:
  for i in account:
    emails = re.findall(r"[A-Za-z0-9\.\-+_]+@[A-Za-z0-9\.\-+_]+\.[A-Za-z]+", i)
    if emails:
      if emails not in email_addresses:
          email_addresses.append(emails)
    if "رقم معروف" in i :
      store_emails_ids.append(i[-5:])
       
for i in range(len(email_addresses)):
  email_addresses[i]= email_addresses[i][0]

# store data in data frames to merge them and then convert them to CVS file. 
storesData = pd.DataFrame({"store_name": store_names, "busenss_type":busenss_typs, 
              "commercial_record": commercial_records,
              "business_chanel": business_chanels, 
              "maroof_number": maroof_numbers, "rating": ratings, 
              "number_raters":number_raters, "about_stores": about_stores})

websites_maroofID = pd.DataFrame({"maroof_numbers_websites": maroof_numbers_websites,
                     "stores_website": stores_websites})

stores_emails=pd.DataFrame({'store_ids':store_emails_ids,
                            'email_addresses': email_addresses})

Both_DFs = pd.merge(websites_maroofID,stores_emails, 
                    how='right',left_on=['maroof_numbers_websites'],
                    right_on=['store_ids'])

stores_DF = pd.merge(Both_DFs,storesData, 
                    how='right',left_on=['store_ids'],
                    right_on=['maroof_number'])
stores_DF.drop('maroof_numbers_websites', inplace=True, axis=1)
stores_DF.drop('store_ids', inplace=True, axis=1)
stores_DF.to_csv("stores_DF.csv")
stores_DF

,stores_website,email_addresses,store_name,busenss_type,commercial_record,business_chanel,maroof_number,rating,number_raters,about_stores
0,NaN,Etian_p@outlook.sa,متجر المنزل,أخرى,4030218247,متجر إلكتروني,1004,0,0,متجر متخصص في بيع و تجهيز المنازل بأدوات النظا...
1,https://istore-ksa.com,info@istore-ksa.com,آي ستور,تسويق إلكتروني,5850037512,متجر إلكتروني,1008,9.6,59,آي ستور موزع معتمد من ابل لبيع منتجات ابل واكس...
2,http://12cups.com.sa,Sales@12cups.com.sa,اثني عشر كوب,أخرى,1010364052,متجر إلكتروني,1013,7.6,7,متخصصون بمختلف انواع القهوة و أدواتها و طرق تح...
3,NaN,almesbah.store@gmail.com,المسباح,أخرى,1010371443,متجر إلكتروني,1016,10,3,أهلاً! نحن المسباح نعمل في مجال السبح وملحقاته...
4,https://tcare-ksa.com/,mashary.software@gmail.com,العناية الفنية لتقنية المعلومات,إلكترونيات واكسسورات,2050100555,متجر إلكتروني,1023,10,1,متخصصون في صيانة جميع انواع أجهزة الكمبيوتر ال...
...,...,...,...,...,...,...,...,...,...,...
58,NaN,golden.style1437@gmail.com,الاستايل الذهبي,مستلزمات المرأة,1132103081,متجر إلكتروني,1449,10,3,فساتين - تيشرتات - بلايز - جزم - شنط\nكل جديد ...
59,NaN,a1s1a@hotmail.com,متجر ألعاب الذكاء,تسويق إلكتروني,4031081040,متجر إلكتروني,1460,0,0,بيع ألعاب تنميه الذكاء والمواهب \nوإكسسوارات ا...
60,NaN,info@sauditoys.org,متجر سعودي تويز مؤسسة الهوايات المبتكرة,أخرى,1010421351,متجر إلكتروني,1478,0,0,متجر متخصص بألعاب الاطفال المتميزه\r\n ...
61,https://glamourbeauty.com,info@glamourbeauty.net,جلامور بيوتي,تسويق إلكتروني,4030276494,متجر إلكتروني,1491,10,370,متجر سعودي مهتم بمستحضرات التجميل والعناية


In [3]:
# Generate Data Profiling file as an EDA (Explonatory Data Analysis)
report = ProfileReport(stores_DF)
report.to_file("EDA_stores_DF.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
# import matplotlib to see the distribution of raters values 
import matplotlib.pyplot as plt

'''
This block of code aims to classify each business type in a single 
container/list, which helps us to identify the most important 
'small businesses in each category. 

The small businesses could be: 
1- Businesses DON'T have websites. 

             OR

2-Businesses have websites but have few customers/raters. 
'''
# Extracting stores that do not have website
stores_noWebsites =  stores_DF[stores_DF['stores_website'].isna()]
print("Number of stores that Do NOT have Website is: "+str(len(stores_noWebsites)))
print("\n")

# Create empty lists as containers
electronics = []
electronics_witoutWebsite = []
digitalMarking = []
digitalMarking_witoutWebsite = []
othrs = []
furniture = []
wemone = []
party = []
e_solution = []
food = []
academic_services = []
salonAndBeauty = []
dsgine_print = []

'''
Extracting Stores that Don't have websites and classifying them
under the business types. 
'''
for i in stores_DF['busenss_type'].unique():

  if i =='إلكترونيات واكسسورات':
    electronics.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in إلكترونيات واكسسورات category: "
    + str(electronics[0]["stores_website"].isna().sum(axis = 0)))

  elif i == 'تسويق إلكتروني': 
    digitalMarking.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in  تسويق إلكتروني category: "
    + str(digitalMarking[0]["stores_website"].isna().sum(axis = 0)))
   
  elif i == 'أخرى':
    othrs.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in أخرى category: "
    + str(othrs[0]["stores_website"].isna().sum(axis = 0)))

  elif i == 'أثاث وديكور': 
    furniture.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in أثاث و ديكور category: "
    + str(furniture[0]["stores_website"].isna().sum(axis = 0)))
    
  elif i == 'مستلزمات المرأة': 
    wemone.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in مسلتزمات المرأة category: "
    + str(wemone[0]["stores_website"].isna().sum(axis = 0)))
    
  elif i == 'تخطيط مناسبات وحفلات':
    party.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in تخطيط مناسبات و حفلات  category: "
    + str(party[0]["stores_website"].isna().sum(axis = 0)))

  elif i == 'حلول إلكترونية': 
    e_solution.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in حلول إلكترونية category: "
    + str(e_solution[0]["stores_website"].isna().sum(axis = 0)))

  elif i == 'مطبخ و مخبوزات': 
    food.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in مطبخ و مخبوزات category: "
    + str(food[0]["stores_website"].isna().sum(axis = 0)))

  elif i == 'خدمات أكاديمية': 
    academic_services.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in خدمات أكاديمية category: "
    + str(academic_services[0]["stores_website"].isna().sum(axis = 0)))

  elif i == 'كوافيرة و تجميل':
    salonAndBeauty.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in كوافيرة و تجميل category: "
    + str(salonAndBeauty[0]["stores_website"].isna().sum(axis = 0)))

  elif i == 'تصميم وطباعة': 
    dsgine_print.append(stores_DF[stores_DF['busenss_type'] == i])
    print("Number of sotres that do NOT have website in تصميم و طباعة category: "
    + str(dsgine_print[0]["stores_website"].isna().sum(axis = 0)))

'''
This code aims to extract stores' emails that under التسويق الإلكتروني 
category, and they do not have websites.  
'''
print("\n")
print("List of Emails of stores Not have Website and its busines type is التسويق الإلكتروني")
print(stores_noWebsites[stores_noWebsites['busenss_type']=='تسويق إلكتروني' ]['email_addresses'])
print("\n")

'''
This code section aims to extract store emails with websites, 
but their raters are fewer than 100.
'''
# Extracting stores that have website but does not have much customers/raters
stores_raters =  stores_DF[stores_DF['stores_website'].notna()].sort_values(by=['number_raters'])
# Convert the column type from str to int 
stores_raters['number_raters'] = stores_raters['number_raters'].astype(int)
plt.hist(stores_raters['number_raters'])
plt.title("The Distribution of Raters")
plt.xlabel("Number of Raters")
plt.ylabel("Number of Stores")
plt.show() 
print("\n")


# Select/extract only stors that have less than 100 raters 
print("Number of stores that have website but the raters is less than 100: "+ 
      str(len(stores_raters[stores_raters['number_raters']< 100])))
print("\n")
Stors_withFewRaters=stores_raters[stores_raters['number_raters']< 100]
print("List of Stores' emails that have few raters: ")
print(Stors_withFewRaters['email_addresses'])
#Stors_withFewRaters


Number of stores that Do NOT have Website is: 23


Number of sotres that do NOT have website in أخرى category: 10
Number of sotres that do NOT have website in  تسويق إلكتروني category: 5
Number of sotres that do NOT have website in إلكترونيات واكسسورات category: 0
Number of sotres that do NOT have website in تخطيط مناسبات و حفلات  category: 3
Number of sotres that do NOT have website in مسلتزمات المرأة category: 4
Number of sotres that do NOT have website in أثاث و ديكور category: 0
Number of sotres that do NOT have website in خدمات أكاديمية category: 0
Number of sotres that do NOT have website in مطبخ و مخبوزات category: 0
Number of sotres that do NOT have website in حلول إلكترونية category: 0
Number of sotres that do NOT have website in تصميم و طباعة category: 1
Number of sotres that do NOT have website in كوافيرة و تجميل category: 0


List of Emails of stores Not have Website and its busines type is التسويق الإلكتروني
22           sa4net@gmail.com
25    rababmobi22@hotmail.com
35   